<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/RAG/LangFuse/LangFuse_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangFuse
Langfuse is an open-source LLM engineering platform  that helps teams collaboratively debug, analyze, and iterate on their LLM applications. All platform features are natively integrated to accelerate the development workflow.

Traces, evals, prompt management and metrics to debug and improve your LLM application.
Onboard via https://langfuse.com

Langfuse helps you build and improve LLM applications across the entire lifecycle:
- Develop: Observability, Langfuse UI & Prompt Management
- Monitor: Traces, Analytics, Metrics & Evaluations
- Test: Experiments, Releases & Datasets

# Links

- https://github.com/langfuse/langfuse
- https://langfuse.com/docs


# Features
- Full context: Capture the complete execution flow including API calls, context, prompts, parallelism and more
- Conversation/session view: In multi-turn conversations, group interactions into sessions
- User tracking: Add your own identifiers to inspect traces from specific users
- Cost tracking: Monitor model usage and costs across your application
- Quality insights: Collect user feedback and identify low-quality outputs
- Low overhead: Designed for production with minimal performance impact
- Best-in-class SDKs: We offer best-in-class SDKs for Python, JS/TS for easy integration
- Framework support: Integrated with popular frameworks like OpenAI SDK, LangChain, and LlamaIndex
- Multi-modal: Support for tracing text, images and other modalities
- Open source: Fully open source with public API for custom integrations

In [1]:
%pip install langfuse langchain langchain_openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.6.2.post1 which is incompatible.


In [2]:
import os
from urllib.request import urlretrieve
from urllib.error import URLError

REPO_URL = "https://github.com/langfuse/langfuse-python"
download_path = "static"
os.makedirs(download_path, exist_ok=True)

test_files = ["puton.jpg", "joke_prompt.wav", "bitcoin.pdf"]
raw_url = f"{REPO_URL}/raw/main/{download_path}"

for file in test_files:
   try:
       urlretrieve(f"{raw_url}/{file}", f"{download_path}/{file}")
       print(f"Successfully downloaded: {file}")
   except URLError as e:
       print(f"Failed to download {file}: {e}")
   except OSError as e:
       print(f"Failed to save {file}: {e}")

Successfully downloaded: puton.jpg
Successfully downloaded: joke_prompt.wav
Successfully downloaded: bitcoin.pdf


In [17]:
from google.colab import userdata
# userdata.get('secretName')
import os
import pprint
# Get keys for your project from the project settings page
# https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get('LANGFUSE_PUBLIC_KEY')
os.environ["LANGFUSE_SECRET_KEY"] =  userdata.get('LANGFUSE_SECRET_KEY')
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Your openai key
os.environ["OPENAI_API_KEY"] = userdata.get('KEY_OPENAI')

In [32]:
from langfuse.openai import openai
import base64

client = openai.OpenAI()

def encode_file(image_path):
    with open(image_path, "rb") as file:
        return base64.b64encode(file.read()).decode("utf-8")

#  OpenAI SDK: Images

In [33]:
content_path = "static/puton.jpg"
content_type = "image/jpeg"

base64_image = encode_file(content_path)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:{content_type};base64,{base64_image}"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.__dict__)

openai.flush_langfuse()

{'id': 'chatcmpl-AZIhXmnAu2lmnwqMt2vVyxj2q16i1', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The image shows a dog with curly fur sitting close to a person's knee, seemingly looking at the camera. There are also a few people in the background, partially visible, but the focus appears to be on the dog. The setting seems to be indoors, on a wooden floor with a colorful rug.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], 'created': 1732976731, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion', 'service_tier': None, 'system_fingerprint': 'fp_3de1288069', 'usage': CompletionUsage(completion_tokens=60, prompt_tokens=25514, total_tokens=25574, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)), '_request_id': 'req

In [35]:
pprint.pprint(response.choices[0].message.content)

("The image shows a dog with curly fur sitting close to a person's knee, "
 'seemingly looking at the camera. There are also a few people in the '
 'background, partially visible, but the focus appears to be on the dog. The '
 'setting seems to be indoors, on a wooden floor with a colorful rug.')


# OpenAI SDK: Audio input and output

In [36]:
content_path = "static/joke_prompt.wav"

base64_string = encode_file(content_path)

response = client.chat.completions.create(
    model="gpt-4o-audio-preview",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Do what this recording says."},
                {
                    "type": "input_audio",
                    "input_audio": {"data": base64_string, "format": "wav"},
                },
            ],
        },
    ],
)

print(response.__dict__)

openai.flush_langfuse()

{'id': 'chatcmpl-AZIjgs6HWnRyIr8uyYQ1DKncg5cxG', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=ChatCompletionAudio(id='audio_674b20e257e481909cffe20adf93a7ee', data=<langfuse.media.LangfuseMedia object at 0x7ecba1f55300>, expires_at=1732980466, transcript='Why don\'t secrets last long in Berlin? Because of all the tall buildings... there\'s no way to keep anything under "über."'), function_call=None, tool_calls=None))], 'created': 1732976864, 'model': 'gpt-4o-audio-preview-2024-10-01', 'object': 'chat.completion', 'service_tier': None, 'system_fingerprint': 'fp_130ac2f073', 'usage': CompletionUsage(completion_tokens=224, prompt_tokens=66, total_tokens=290, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=185, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=39), prompt_tokens_details=PromptTokensDetails(audio_tokens=49, cached_tok

In [37]:
pprint.pprint(response.choices[0].message.audio.transcript)

("Why don't secrets last long in Berlin? Because of all the tall buildings... "
 'there\'s no way to keep anything under "über."')


# Python Decorator: Attachments via LangfuseMedia

In [38]:
from langfuse.decorators import observe, langfuse_context
from langfuse.media import LangfuseMedia

with open("static/bitcoin.pdf", "rb") as pdf_file:
        pdf_bytes = pdf_file.read()

wrapped_obj = LangfuseMedia(
    obj=pdf_bytes, content_bytes=pdf_bytes, content_type="application/pdf"
)

@observe()
def main():
    langfuse_context.update_current_trace(
        metadata={
            "context": wrapped_obj
        },
    )

    return # Limitation: LangfuseMedia object does not work in decorated function IO

main()

langfuse_context.flush()

# Langchain: Image input

In [39]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

from langfuse.callback import CallbackHandler

handler = CallbackHandler()
model = ChatOpenAI(model="gpt-4o-mini")

image_data = encode_file("static/puton.jpg")

message = HumanMessage(
    content=[
        {"type": "text", "text": "What's in this image?"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)

response = model.invoke([message], config={"callbacks": [handler]})

print(response.content)

handler.flush()

The image shows a dog with a curly coat sitting in front of a person, with its paws resting on the person's knee. The dog appears to be happy, with its tongue out. In the background, there are several people standing, and the setting looks like a cozy indoor space with wooden flooring and a colorful rug.


In [40]:
pprint.pprint(response.content)

('The image shows a dog with a curly coat sitting in front of a person, with '
 "its paws resting on the person's knee. The dog appears to be happy, with its "
 'tongue out. In the background, there are several people standing, and the '
 'setting looks like a cozy indoor space with wooden flooring and a colorful '
 'rug.')


In [21]:
%pip install llama-index langfuse llama-index-vector-stores-milvus  --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.2 MB/s eta 0:00:00


# LlamaIndex & Milvus Integration

In [41]:
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager
from langfuse.llama_index import LlamaIndexCallbackHandler

langfuse_callback_handler = LlamaIndexCallbackHandler()
Settings.callback_manager = CallbackManager([langfuse_callback_handler])

In [42]:
pdf_path = "/content/drive/MyDrive/data/pdfs"

In [43]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# load documents
data = SimpleDirectoryReader(input_dir=pdf_path,required_exts=[".pdf"]).load_data()

In [44]:
len(data)

143

In [45]:
data[1]

Document(id_='e9e11c1c-1858-4dbc-84ef-d3e60c8ddf20', embedding=None, metadata={'page_label': '2', 'file_name': 'Parser Source 2.pdf', 'file_path': '/content/drive/MyDrive/data/pdfs/Parser Source 2.pdf', 'file_type': 'application/pdf', 'file_size': 1930169, 'creation_date': '2024-11-30', 'last_modified_date': '2024-11-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="Aardvark Constructions Limited \nDate \nCompany Profile \nCompany Number \n09/08/2024 123456 \n \nDate 09/08/2024 AARDVCONST/MCPARTLA-C Page 1 \nMain Details \n Name: Aardvark Constructions Limited \n QuickRef: AARDVCONST \n Country: United Kingdom \n Company Number: 123456 \n Incorporated: 20/10/2020 \n Company Type: Limited by

In [27]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore


vector_store = MilvusVectorStore(
    uri="/content/drive/MyDrive/data/pdfs/milvus/milvus.db", dim=1536, overwrite=False
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 4a426853b2094d2d8264c43efac1bcc8
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: llamacollection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: llamacollection


In [28]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    data, storage_context=storage_context
)

In [46]:
prompt = """
Provide Main Details of the company Aardvark Constructions Limited. Including following details:
Name:
Country:
Company Number:
Incorporated:
Company Type:
Company Status:
Primary Addresses Registered Office:
Accounting Dates:
Confirmation Statement:
"""

In [47]:
response = index.as_query_engine().query(prompt)
pprint.pprint(response)

Response(response='Name: Aardvark Constructions Limited\n'
                  'Country: United Kingdom\n'
                  'Company Number: 123456\n'
                  'Incorporated: 20/10/2020\n'
                  'Company Type: Limited by Shares\n'
                  'Company Status: Active\n'
                  'Primary Addresses Registered Office: 6 Chancery Road, '
                  'London, WC2A 5DP, United Kingdom\n'
                  'Accounting Dates: Last Period End - 16/11/2022, Current '
                  'Period End - 16/11/2024\n'
                  'Confirmation Statement: Last Signed Filed - 17/02/2023, '
                  'Next Overdue - 03/03/2023',
         source_nodes=[NodeWithScore(node=TextNode(id_='575f7837-3d73-45df-a958-7528f8723ecd', embedding=None, metadata={'page_label': '2', 'file_name': 'Parser Source 2.pdf', 'file_path': '/content/drive/MyDrive/data/pdfs/Parser Source 2.pdf', 'file_type': 'application/pdf', 'file_size': 1930169, 'creation_date': '2024-11-07

In [48]:
# As we want to immediately see result in Langfuse, we need to flush the callback handler
langfuse_callback_handler.flush()